In [ ]:
! pip install datasets

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
encoded = tokenizer.encode("Do not meddle in the affairs of wizards")

In [ ]:
# prompt: write python code to print the textual tokens in sequential order from a string, using the above tokenizer

print("Encoded text:", tokenizer.convert_ids_to_tokens(encoded))


In [ ]:
print(encoded)

In [ ]:
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
print(encoded_input.keys())
print(encoded_input['input_ids'])

In [ ]:
! pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

In [ ]:
words = ["quick", "fast", "red", "blue", "ferari"]
single_word_embeddings = model.encode(words)

for word, embed in zip(words, single_word_embeddings):
  print("word: ", word)
  print("embed: ", embed[0:10])
  print("")

In [ ]:
# prompt: python code to compute the matrix of cosines between all of the pairs of words in the list above.

from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(single_word_embeddings)
print(cos_sim)


In [ ]:
from scipy.spatial.distance import cosine
for sentence in sentences:
    print("Sentence:", sentence)
print("")
print("Cosine similarity between the first two sentences:", cosine(embeddings[0], embeddings[1]))
print("Cosine similarity between the second and third sentences:", cosine(embeddings[1], embeddings[2]))
print("Cosine similarity between the first and third sentences:", cosine(embeddings[0], embeddings[2]))

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load the pretrained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Disable gradient calculation for inference
with torch.no_grad():
    # Sentence to encode
    input_text = "A fly can fly"

    # Tokenize the input text and convert to tensor
    encoded_input = tokenizer(input_text, return_tensors='pt')

    # Get the model's output (hidden states)
    outputs = model(**encoded_input)
    last_hidden_states = outputs.last_hidden_state

    # Get all token ids to skip special tokens if desired
    token_ids = encoded_input.input_ids[0].tolist()

    # Print the embeddings for each token in the sentence
    for i, token_id in enumerate(token_ids):
      token = tokenizer.decode([token_id])
      embedding = last_hidden_states[0, i]
      print(f"Token: {token} - Embedding: {embedding}")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine

# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Define the sentences
sentence1 = "Time flies like an arrow"
sentence2 = "Fruit flies like a banana"

# Tokenize and encode the sentences
encoded_sentence1 = tokenizer(sentence1, return_tensors='pt')
encoded_sentence2 = tokenizer(sentence2, return_tensors='pt')

# Identify the 'like' token index
word = "like"
word_index1 = tokenizer.encode(sentence1, add_special_tokens=True).index(tokenizer.encode(word)[1]) - 1
word_index2 = tokenizer.encode(sentence2, add_special_tokens=True).index(tokenizer.encode(word)[1]) - 1

# Get the embeddings for each encoded sentence
with torch.no_grad(): # Disable gradient tracking
    output_sentence1 = model(**encoded_sentence1)
    output_sentence2 = model(**encoded_sentence2)

# Extract the embeddings for the word 'like' from both sentences using their indices
embedding_sentence1 = output_sentence1.last_hidden_state[0, word_index1]
embedding_sentence2 = output_sentence2.last_hidden_state[0, word_index2]

# Calculate the cosine similarity
similarity = 1 - cosine(embedding_sentence1.detach().numpy(), embedding_sentence2.detach().numpy())

# Print the results
print("Cosine Similarity:", similarity)